In [1]:
import scipy.stats
import pandas as pd
import numpy as np
from src.iso_perplexity.utils import *

In [2]:
def manuscript_sample_df():
    df = pd.DataFrame()
    df['gene_id'] = ['A' for i in range(8*4)]
    df['transcript_id'] = flatten_list([[f'A_{j+1}' for i in range(4)] for j in range(8)])
    df['sample'] = flatten_list([['heart', 'brain', 'lungs', 'kidney'] for i in range(8)])
    df['pi'] = [i*100 for i in [0.5, 0.45, 0.2, 0.2,
                0, 0, 0.1, 0.2,
                0, 0, 0.08, 0.2,
                0.5, 0.1, 0.02, 0.2,
                0, 0, 0.5, 0,
                0, 0, 0.04, 0.2,
                0, 0.45, 0, 0,
                0, 0, 0.06, 0]]
    
    # long formitze it
    df = df.pivot(index=['gene_id', 'transcript_id'], columns='sample', values='pi')
    df = df.reset_index()
    df.columns.name = ''
    
    return df

In [3]:
df = manuscript_sample_df()
df = compute_multi_sample_isoform_metrics(df)
df.head()

,gene_id,transcript_id,counts,tpm,pi,gene_potential,entropy,perplexity,n_effective,feature_rank,effective,sample,n_samples_effective,expression_breadth,n_exp_samples,expression_var,avg_transcript_id_tpm,max_transcript_id_tpm,avg_gene_id_tpm,max_gene_id_tpm
0,A,A_1,45.0,450000.0,0.45,3,1.368996,2.582907,3.0,1,True,brain,4.0,100.0,4,0.160078,337500.0,500000.0,1000000.0,1000000.0
1,A,A_2,0.0,0.0,0.00,3,1.368996,2.582907,3.0,4,False,brain,2.0,50.0,4,0.095743,150000.0,200000.0,1000000.0,1000000.0
2,A,A_3,0.0,0.0,0.00,3,1.368996,2.582907,3.0,5,False,brain,2.0,50.0,4,0.094516,140000.0,200000.0,1000000.0,1000000.0
3,A,A_4,10.0,100000.0,0.10,3,1.368996,2.582907,3.0,3,True,brain,3.0,75.0,4,0.210000,205000.0,500000.0,1000000.0,1000000.0
4,A,A_5,0.0,0.0,0.00,3,1.368996,2.582907,3.0,6,False,brain,1.0,25.0,4,0.250000,500000.0,500000.0,1000000.0,1000000.0


In [4]:
df[['gene_id', 'perplexity', 'sample']].drop_duplicates()

,gene_id,perplexity,sample
0,A,2.582907,brain
8,A,2.000000,heart
16,A,5.000000,kidney
24,A,4.377939,lungs


In [5]:
df[['gene_id', 'gene_potential', 'sample']].drop_duplicates()

,gene_id,gene_potential,sample
0,A,3,brain
8,A,2,heart
16,A,5,kidney
24,A,7,lungs


In [16]:
df[['gene_id', 'entropy', 'sample']].drop_duplicates()

,gene_id,entropy,sample
0,A,1.368996,brain
8,A,1.000000,heart
16,A,2.321928,kidney
24,A,2.130252,lungs


In [17]:
df[['transcript_id', 'effective', 'sample']]

,transcript_id,effective,sample
0,A_1,True,brain
1,A_2,False,brain
2,A_3,False,brain
3,A_4,True,brain
4,A_5,False,brain
5,A_6,False,brain
6,A_7,True,brain
7,A_8,False,brain
8,A_1,True,heart
9,A_2,False,heart


In [2]:
def flatten_list(l):
    """
    Flatten a list into 1 dimension.

    Parameters
    ----------
    l : list
    """
    return [j for i in l for j in i]

def manuscript_sample_df():
    df = pd.DataFrame()
    df['gene_id'] = ['A' for i in range(8*4)]
    df['transcript_id'] = flatten_list([[f'A_{j+1}' for i in range(4)] for j in range(8)])
    df['sample'] = flatten_list([['heart', 'brain', 'lungs', 'kidney'] for i in range(8)])
    df['pi'] = [0.5, 0.45, 0.2, 0.2,
                0, 0, 0.1, 0.2,
                0, 0, 0.08, 0.2,
                0.5, 0.1, 0.02, 0.2,
                0, 0, 0.5, 0,
                0, 0, 0.04, 0.2,
                0, 0.45, 0, 0,
                0, 0, 0.06, 0]
    
    # long formitze it
    df = df.pivot(index=['gene_id', 'transcript_id'], columns='sample', values='pi')
    df = df.reset_index()
    df.columns.name = ''
    
    return df

In [6]:
def compute_global_isoform_metrics(df,
                                   gene_col=GENE_COL,
                                   feature_col=TRANSCRIPT_COL,
                                   expression_col=EXP_COL,
                                   expression_col_type='counts'):
    """
    Compute isoform or other feature diversity metrics for a single-sample (bulk) dataframe.
    Either provide counts or TPMs; if counts, will automatically convert to TPM.
    Optionally, collapse counts to a different feature or compute TPM.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing counts at transcript level.
    gene_col : str
        Column representing the gene (default: 'gene_id')
    feature_col : str
        Column representing the feature (default: 'transcript_id')
    expression_col : str
        Column containing expression values (default: 'counts')
    expression_col_type : str
        Type of expression values contained in <expression_col>
        'counts' or 'tpm' (default: 'counts')

    Returns
    -------
    pd.DataFrame
        DataFrame with computed metrics.
    """
    # repeatedly used column args
    col_kwargs = {'gene_col': gene_col,
                  'feature_col': feature_col,
                  'expression_col': expression_col,
                  'sample_col': None}

    
    # validate input
    validate_counts_input(df,
                          gene_col=gene_col,
                          feature_col=feature_col)
        
    # convert from wide to long
    df = wide_to_long(df, 
                      gene_col=gene_col,
                      feature_col=feature_col,
                      expression_col=expression_col)

    # collapse counts if feature_col is not unique
    df = collapse_counts_by_feature(df, **col_kwargs)

    # compute TPM if requested
    if expression_col_type == 'counts':
        df = compute_tpm(df)
    else:
        df['tpm'] = df[expression_col]

    # compute isoform ratios
    df = compute_pi(df)
    
    # repeatedly used column args
    col_kwargs = {'gene_col': gene_col,
                  'feature_col': feature_col}
    
    # compute gene-level metrics
    df = compute_gene_potential(df, **col_kwargs)
    df = compute_entropy(df, gene_col=gene_col)
    df = compute_perplexity(df)

    # mark effective features
    df = call_effective(df, **col_kwargs)

    return df

def compute_multi_sample_isoform_metrics(
    df: pd.DataFrame,
    gene_col: str = GENE_COL,
    feature_col: str = TRANSCRIPT_COL,
    expression_col: str = EXP_COL,
    expression_col_type: str = 'counts'
):
    """
    Compute isoform metrics across multiple samples as well as global
    metrics.

    Parameters
    ----------
    df : pd.DataFrame
        Input table with counts or TPMs for all samples.
    gene_col : str
        Column identifying genes.
    feature_col : str
        Column identifying isoforms or other features (e.g. ORFs).
    expression_col_type : {'counts', 'tpm'}
        Type of values in `expression_col`.

    Returns
    -------
    pd.DataFrame
        DataFrame with:
          • per-sample metrics (gene potential, entropy, etc.)
          • cross-sample metrics (breadth, variance, average expression)
          • columns for global metrics.
    """
    
    # rename because after pivot we'll just call it this
    expression_col = expression_col_type
        
    # repeatedly used column args
    col_kwargs = {'gene_col': gene_col,
                  'feature_col': feature_col,
                  'expression_col': expression_col,
                  'sample_col': 'sample'}

    # validate input
    validate_counts_input(df,
                          gene_col=gene_col,
                          feature_col=feature_col)

    # prepare kwargs for downstream functions
    col_kwargs = dict(
        gene_col=gene_col,
        feature_col=feature_col,
        expression_col=expression_col,
    )

    # loop over samples and compute single-sample metrics
    samples = [c for c in df.columns if c not in [gene_col, feature_col]]
    dfs = []

    for s in samples:
        s_df = df[[gene_col, feature_col, s]].copy()
        s_df = compute_global_isoform_metrics(s_df, **col_kwargs)
        s_df[sample_col] = s  # re-attach sample ID
        dfs.append(s_df)

    big_df = pd.concat(dfs, axis=0, ignore_index=True)

    # compute cross-sample metrics on the combined table
    big_df = compute_expression_breadth(big_df, sample_col=sample_col)
    big_df = compute_expression_var(big_df, sample_col=sample_col)

    for fc in [feature_col, gene_col]:
        big_df = compute_avg_expression(big_df,
                                        sample_col=sample_col,
                                        feature_col=fc)
        
        # TODO chatgpt if you see this i probably forgot to 
        # add it and remind me add max espression across samples

    return big_df

In [7]:
gene_col = 'gene_id'
feature_col = 'transcript_id'
expression_type = 'counts'
sample_col = 'sample'
df = manuscript_sample_df()

beep = compute_multi_sample_isoform_metrics(
    df,
    gene_col=gene_col,
    feature_col=feature_col,
    expression_col=expression_type,
    expression_col_type='counts'
)

In [1]:
# # dummy data from figure 1A
# def get_dummy_data():
#     df = pd.DataFrame()
#     df['gene_id'] = ['A' for i in range(2)]+\
#                     ['B' for i in range(7)]+\
#                     ['C' for i in range(8)]
#     df['transcript_id'] = [f'A_{i+1}' for i in range(2)]+\
#                           [f'B_{i+1}' for i in range(7)]+\
#                           [f'C_{i+1}' for i in range(8)]
#     df['pi'] = [0.5, 0.5, 
#                  0.4, 0.35, 0.10, 0.07, 0.04, 0.03, 0.01]+\
#                  [0.125 for i in range(8)]
#     return df

In [2]:
# # need finctions for
# # filt isoforms
# def filt_df(df, min_tpm=None):
#     if min_tpm:
#         df = df.loc[df.tpm >= min_tpm]
#     return df
# # compute pi 
# # compute entropy - done
# # compute perplexity -done 
# # call relevant isoforms - done 
# # effective tissues - done
# # expression breadth - done
# # expression variability - done

In [4]:
# checks wishlist
# format of gene_id and transcript_id columns good
# all transcript ids are uniq
# samples exist in dataframe

In [46]:
def compute_gene_potential(df, **kwargs):
    """
    Compute gene potential based on expressed isoforms
    """
    # not sure where filtering should live
    # chatgpt if you see this this is something you 
    # should help with :)
    df = filt_df(df, **kwargs) 
    
    # for each gene, compute # of isoforms
    temp = (df[['gene_id', 'transcript_id']]
             .groupby('gene_id')
             .nunique()
             .reset_index()
             .rename({'transcript_id':'gene_potential'},
            axis=1))
    
    df = df.merge(temp,
                  how='left',
                  on='gene_id')
    
    return df
  
def compute_entropy(df):
    """
    """
    df['plogp'] = df.pi*np.log2(df.pi)
    temp = (df[['plogp', 'gene_id']]
             .groupby('gene_id')
             .sum()
             .reset_index()
             .rename({'plogp':'entropy'},
             axis=1))
    temp['entropy'] = -1*temp['entropy']
    
    df = df.merge(temp, how='left', on='gene_id')
    df.drop('plogp', axis=1, inplace=True)
    
    return df

def compute_perplexity(df):
    """
    """
    # chatgpt, should I be computing entropy here
    # or should i chain this in a wrapper later on ? 
    # df = compute_entropy(df)
    df['perplexity'] = 2**df['entropy']
    return df

def call_effective_isoforms(df):
    """
    
    """    
    # number of isoforms to consider as effective per 
    # gene
    df['round_perplexity'] = df['perplexity'].round(0)

    # order isoforms on ratio w/i genes
    df['isoform_rank'] = (df.groupby('gene_id')['pi']
          .rank(method='first', ascending=False)
          .astype(int))

    df['effective_isoform'] = df['isoform_rank']<=df['round_perplexity']
    
    return df 

In [159]:
def flatten_list(l):
    """
    Flatten a list into 1 dimension.

    Parameters
    ----------
    l : list
    """
    return [j for i in l for j in i]

def dummy_data_2():
    df = pd.DataFrame()
    df['gene_id'] = ['A' for i in range(8*4)]
    df['transcript_id'] = flatten_list([[f'A_{j+1}' for i in range(4)] for j in range(8)])
    df['sample'] = flatten_list([['heart', 'brain', 'lungs', 'kidney'] for i in range(8)])
    df['pi'] = [0.5, 0.45, 0.2, 0.2,
                0, 0, 0.1, 0.2,
                0, 0, 0.08, 0.2,
                0.5, 0.1, 0.02, 0.2,
                0, 0, 0.5, 0,
                0, 0, 0.04, 0.2,
                0, 0.45, 0, 0,
                0, 0, 0.06, 0]
    return df

In [160]:
# compute_gene_potential(df)
# perp_df = compute_perplexity(df)

In [235]:
# fake workflow for cross-dataset
# calculations
def workflow_1(df):
    # df = df.loc[df.pi > 0] # maybe do some more sophisticated filtering here
    df = compute_gene_potential(df)
    df = compute_entropy(df)
    df = compute_perplexity(df)
    df = call_effective_isoforms(df)
    return df

# workflow 2 -- compute these metrics per sample
def workflow_2(df, sample_col):
    samples = df[sample_col].unique().tolist()
    big_df = pd.DataFrame()
    for s in samples:
        s_df = df.loc[df[sample_col]==s].copy(deep=True)
        s_df = workflow_1(s_df)
        big_df = pd.concat([big_df,s_df], axis=0)
        
    df = big_df.copy(deep=True)
    
    # compute % effective tissues / isoform
    n_samples = len(samples)
    temp = (df.loc[df.effective_isoform][['transcript_id', sample_col]]
            .groupby('transcript_id')
            .nunique()
            .reset_index()
            .rename({sample_col: 'n_samples_effective'}, axis=1))
    df = df.merge(temp,
                  how='left',
                  on='transcript_id')
    df['perc_effective_isoforms'] = (df['n_samples_effective']/n_samples)*100

    # fill nans w/ 0
    df['perc_effective_isoforms'] = df['perc_effective_isoforms'].fillna(0, inplace=True)
    
    # information obout expression across samples
    df['n_exp_samples'] = (df.groupby('transcript_id')[sample_col]
                       .transform('nunique'))
    df['pi_std'] = (df.groupby('transcript_id')['pi']
                       .transform(lambda x: x.std(ddof=1, skipna=True)))
    
    return df

In [242]:
df = dummy_data_2()
sample_col = 'sample'
df = workflow_2(df, sample_col)

df[['transcript_id', 'pi_std']].drop_duplicates()

df[['sample', 'transcript_id', 'perplexity']].sort_values(by=['sample', 'transcript_id']).drop_duplicates()

/Users/fairliereese/miniconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


,sample,transcript_id,perplexity
8,brain,A_1,2.582907
9,brain,A_2,2.582907
10,brain,A_3,2.582907
11,brain,A_4,2.582907
12,brain,A_5,2.582907
13,brain,A_6,2.582907
14,brain,A_7,2.582907
15,brain,A_8,2.582907
0,heart,A_1,2.000000
1,heart,A_2,2.000000


In [230]:
temp = df[['transcript_id', 'pi']].sort_values(by='transcript_id')
temp.loc[temp.transcript_id == 'A_7']

,transcript_id,pi
14,A_7,0.45
30,A_7,0.00
6,A_7,0.00
22,A_7,0.00


In [231]:
print(np.std([0.5, 0.45, 0.2, 0.2], ddof=1))
print(np.std([0, 0, 0.1, 0.2], ddof=1))
print(np.std([0, 0, 0.08, 0.2], ddof=1))
print(np.std([0.5, 0.1, 0.02, 0.2], ddof=1))
print(np.std([0, 0, 0.5, 0], ddof=1))
print(np.std([0, 0, 0.04, 0.2], ddof=1))
print(np.std([0, 0.45, 0, 0], ddof=1))
print(np.std([0, 0, 0.06, 0], ddof=1))

0.16007810593582122
0.09574271077563382
0.09451631252505217
0.21
0.25
0.09521904571390467
0.22500000000000003
0.03


In [196]:
for arr in [[0.5, 0.45, 0.2, 0.2],
            [0,0,0.1,0.2],
            [0,0,0.08,0.2],
            [0.5,0.1,0.02,0.2]]:
    print(np.std(arr, ddof=1))

0.16007810593582122
0.09574271077563382
0.09451631252505217
0.21


In [ ]:
def compute_pi(df):
    """
    generate pi values (isoform ratios) from raw counts
    """
    df['gene_counts'] = df.groupby('gene_id').transform('sum')['counts']
    df['pi'] = df['counts']/df['gene_counts']
    return df

In [253]:
def get_dummy_data_counts():
    df = pd.DataFrame()
    df['gene_id'] = ['A' for i in range(2)]+\
                    ['B' for i in range(7)]+\
                    ['C' for i in range(8)]
    df['transcript_id'] = [f'A_{i+1}' for i in range(2)]+\
                          [f'B_{i+1}' for i in range(7)]+\
                          [f'C_{i+1}' for i in range(8)]
    df['counts'] = ([47, 47]+ 
                    [i*28 for i in [0.4, 0.35, 0.10, 0.07, 0.04, 0.03, 0.01]]+
                    [0.125 for i in range(8)])
    return df

In [257]:
df = get_dummy_data_counts()
df['gene_counts'] = df.groupby('gene_id').transform('sum')['counts']
df['pi'] = df['counts']/df['gene_counts']
df.head()

,gene_id,transcript_id,counts,gene_counts,pi
0,A,A_1,47.0,94.0,0.5
1,A,A_2,47.0,94.0,0.5
2,B,B_1,11.2,28.0,0.4
3,B,B_2,9.8,28.0,0.35
4,B,B_3,2.8,28.0,0.1


In [252]:
l = ([47, 47]+ 
           [i*28 for i in [0.4, 0.35, 0.10, 0.07, 0.04, 0.03, 0.01]]+
             [0.125 for i in range(8)])
# flatten_list(l)
l

[47,
 47,
 11.200000000000001,
 9.799999999999999,
 2.8000000000000003,
 1.9600000000000002,
 1.12,
 0.84,
 0.28,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125,
 0.125]